### 1. Declare imports and file paths

In [1]:
import os
import pandas as pd
import math

INPUT_DIR = "input"
OUTPUT_DIR = "output"

VEDDRA_FILE = os.path.join(OUTPUT_DIR, 'veddra.csv')

PRODUCTS = ['librela', 'solensia']
INPUT_DIR_EVVET = os.path.join(INPUT_DIR, 'evvet')
INPUT_DIR_EVVET_PRODUCTS = [os.path.join(INPUT_DIR_EVVET, product) for product in PRODUCTS]

### 2. Create veddra lookup tables

In [2]:
veddra = pd.read_csv(VEDDRA_FILE)

# Create a dictionary for O(1) lookup
veddra_lookup = {}

# Populate the dictionary with PT and LLT as keys
for _, row in veddra.iterrows():
    pt = row['Current Preferred Term (PT)']
    llt = row['Current Low Level Term (LLT)']
    soc = row['Current System Organ Class (SOC) Term']
    hlt = row['Current High Level Term (HLT)']
    
    # Add PT to the dictionary
    veddra_lookup[pt] = (soc, hlt)
    
    # Add LLT to the dictionary
    veddra_lookup[llt] = (soc, hlt)

display(veddra_lookup['Uveitis'])

('Eye disorders', 'Iris, ciliary body and choroid disorders')

### 3. Generate all output files!

In [3]:
import datetime
import json
import os
import pandas as pd
import pytz

def generate(PRODUCT):
    INPUT_DIR_EVVET = os.path.join(INPUT_DIR, 'evvet', PRODUCT)
    OUTPUT_DIR_EVVET = os.path.join(OUTPUT_DIR, 'evvet', PRODUCT)
    OUTPUT_EVVET_FILE = os.path.join(OUTPUT_DIR_EVVET, f"{PRODUCT}.csv")
    OUTPUT_EVVET_META = os.path.join(OUTPUT_DIR_EVVET, f"{PRODUCT}_meta.json")

    # Define the PDT timezone
    pdt = pytz.timezone('America/Los_Angeles')
    
    print()
    print(f"===========================================")
    print(f"Processing {PRODUCT.capitalize()} EVVet data...")
    print(f"===========================================")
    print()

    # Get the current date and time in UTC and convert to PDT
    current_datetime = datetime.datetime.now(pytz.utc).astimezone(pdt)
    last_updated = current_datetime.strftime("%B %d, %Y %H:%M:%S %Z")
    print(f"Last run:\n{last_updated}")

    # # Delete the output file if it exists
    # if os.path.exists(OUTPUT_EVVET_FILE):
    #     os.remove(OUTPUT_EVVET_FILE)

    # List all CSV files in the input directory
    csv_files = [f for f in os.listdir(INPUT_DIR_EVVET) if f.endswith('.csv')]
    
    df_list = []
    for file in csv_files:
        df = pd.read_csv(os.path.join(INPUT_DIR_EVVET, file))
        df.dropna(how='all', inplace=True)
        print(f"\nProcessing file: {file}...")
        print(f"Number of rows: {len(df)}")
        df_list.append(df)
    
    # Concatenate all dataframes into one
    master_df = pd.concat(df_list, ignore_index=True)
    master_df.drop_duplicates(inplace=True)

    # Reorder columns to move 'AER form' to the last index if it exists in the dataframe
    if 'AER form' in master_df.columns:
        cols = master_df.columns.tolist()
        cols.append(cols.pop(cols.index('AER form')))
        master_df = master_df[cols]

    # Move 'Drug' column to the end if it exists in the dataframe
    if 'Drug' in master_df.columns:
        cols = master_df.columns.tolist()
        cols.append(cols.pop(cols.index('Drug')))
        master_df = master_df[cols]

    # Move 'Received date' column to the first index if it exists in the dataframe
    if 'Received date' in master_df.columns:
        cols = master_df.columns.tolist()
        cols.insert(0, cols.pop(cols.index('Received date')))
        master_df = master_df[cols]

    # Change Received Date column's format
    master_df['Received date'] = pd.to_datetime(master_df['Received date'])

    # Set the index to 'Received date' for master_df
    master_df.set_index('Received date', inplace=True)

    # Sort the dataframe by 'Received date'
    master_df.sort_values(by='Received date', inplace=True)

    # Drop any empty rows in master_df
    master_df.dropna(how='all', inplace=True)

    # Check for any rows where dates couldn't be parsed
    if master_df.index.isna().any():
        print("Some dates couldn't be parsed and were set to NaT")

    #####
    # Add VeDDRA SOC and HLT columns to the evvet DataFrame
    #####
    # Initialize the columns
    master_df['VeDDRA SOC'] = ''
    master_df['VeDDRA HLT'] = ''

    # Define a function to process each row
    def process_row(row):
        reactions = row['Reaction'].split(',')
        soc_matches = set()
        hlt_matches = set()
        for reaction in reactions:
            reaction = reaction.strip()
            if reaction in veddra_lookup:
                soc, hlt = veddra_lookup[reaction]
                soc_matches.add(soc)
                hlt_matches.add(hlt)
        
        row['VeDDRA SOC'] = '; '.join(sorted(soc_matches)) if soc_matches else ''
        row['VeDDRA HLT'] = '; '.join(sorted(hlt_matches)) if hlt_matches else ''
        return row

    # Apply the function to each row
    master_df = master_df.apply(process_row, axis=1)

    # Remove the columns from their current positions
    veddra_soc = master_df.pop('VeDDRA SOC')
    veddra_hlt = master_df.pop('VeDDRA HLT')
    # Insert the columns at the desired positions
    master_df.insert(11, 'VeDDRA SOC', veddra_soc)
    master_df.insert(12, 'VeDDRA HLT', veddra_hlt)

    print()
    print("VeDDRA SOC and HLT columns added to the dataframe...")

    # Load the most recent CSV listed in the meta file
    with open(OUTPUT_EVVET_META, "r") as f:
        meta = json.load(f)
    
    if meta['csvs']:
        last_csv_info = meta['csvs'][0]
        last_master_df = pd.read_csv(os.path.join(OUTPUT_DIR_EVVET, last_csv_info['name']), index_col='Received date', parse_dates=True)
        

        # Fill NaN values with an empty string to ensure that "NaN" and empty don't trigger a difference
        master_df[['VeDDRA SOC', 'VeDDRA HLT']] = master_df[['VeDDRA SOC', 'VeDDRA HLT']].fillna('')
        last_master_df[['VeDDRA SOC', 'VeDDRA HLT']] = last_master_df[['VeDDRA SOC', 'VeDDRA HLT']].fillna('')

        # Check if the dataframes are identical
        if master_df.equals(last_master_df):
            print()
            print("Duplicate fetch. Aborted!")
            return last_master_df
        else: 
            print()
            print("New data found. Proceeding with the update...")
            
    # Write the updated master dataframe to a new CSV
    new_csv_name = f"{PRODUCT}_{current_datetime.strftime('%Y%m%d')}.csv"
    # Write to csv archive
    master_df.to_csv(os.path.join(OUTPUT_DIR_EVVET, new_csv_name))
    # Make this the new master file (copy)
    master_df.to_csv(OUTPUT_EVVET_FILE)
    
    print("\nMaster file compiled and written to archive and source.")

    print()
    print(f"Total cases: {len(master_df)}")
    print(f"Animals affected: {master_df['Animals affected'].sum()}")
    print(f"Animals treated: {master_df['Animals treated'].sum()}")
    print(f"Animals died: {master_df['Animals died'].sum()}")

    print()

    # Update the meta file
    new_csv_info = {
        "id": last_csv_info['id'] + 1 if meta['csvs'] else 1,
        "name": new_csv_name,
        "timestamp": current_datetime.isoformat()
    }
    meta['csvs'].insert(0, new_csv_info)
    
    with open(OUTPUT_EVVET_META, "w") as f:
        json.dump(meta, f, indent=4)
    print(f'{OUTPUT_EVVET_META} updated!')

    return master_df

evvet = {}

for PRODUCT in PRODUCTS:
    evvet[PRODUCT] = generate(PRODUCT)


Processing Librela EVVet data...

Last run:
February 06, 2025 20:58:57 PST

Processing file: librela-2025.csv...
Number of rows: 1562

Processing file: librela-2024.csv...
Number of rows: 15289

Processing file: librela-2023.csv...
Number of rows: 5105

Processing file: librela-2022.csv...
Number of rows: 3169

Processing file: librela-2021.csv...
Number of rows: 383

VeDDRA SOC and HLT columns added to the dataframe...

New data found. Proceeding with the update...

Master file compiled and written to archive and source.

Total cases: 25508
Animals affected: 26617
Animals treated: 30485.0
Animals died: 3335.0

output/evvet/librela/librela_meta.json updated!

Processing Solensia EVVet data...

Last run:
February 06, 2025 20:58:58 PST

Processing file: solensia-2023.csv...
Number of rows: 4538

Processing file: solensia-2022.csv...
Number of rows: 1696

Processing file: solensia-2021.csv...
Number of rows: 67

Processing file: solensia-2025.csv...
Number of rows: 397

Processing file: 

### 4. Proportional Librela back-distribution of Feb 2022 Reporting Delay (Beta)

In [ ]:
try:
    data = evvet.copy()
    # data.set_index('Received date', inplace=True)
    # data['Received date'] = pd.to_datetime(data['Received date'])

    # Group data and reset index to handle aggregation properly
    aggregated_data = data.groupby('Received date').agg(
        Animals_affected=('Animals affected', 'sum'),
        Animals_died=('Animals died', 'sum')
    ).reset_index()

    # Rename columns to remove underscores
    aggregated_data.rename(columns={'Animals_affected': 'Animals affected', 'Animals_died': 'Animals died'}, inplace=True)

    # Now set 'Received date' as index for date-based operations
    aggregated_data.set_index('Received date', inplace=True)

    # Store original data for verification and retain original columns
    original_totals = {
        'rows': len(aggregated_data),
        'Animals affected': aggregated_data['Animals affected'].sum(),
        'Animals died': aggregated_data['Animals died'].sum()
    }
    aggregated_data['Animals affected original'] = aggregated_data['Animals affected']
    aggregated_data['Animals died original'] = aggregated_data['Animals died']

    start_date = pd.to_datetime('2021-02-08')
    # Define outlier dates and their respective redistribution periods
    outlier_dates = pd.to_datetime(['2022-02-10', '2022-02-11', '2022-07-21', '2023-01-19', '2024-05-06'])
    redistribution_periods = {
        outlier_dates[0]: (start_date, outlier_dates[0] - pd.Timedelta(days=1)),
        outlier_dates[1]: (start_date, outlier_dates[1] - pd.Timedelta(days=1)),
        outlier_dates[2]: (outlier_dates[1] + pd.Timedelta(days=1), outlier_dates[2] - pd.Timedelta(days=1)),
        outlier_dates[3]: (outlier_dates[2] + pd.Timedelta(days=1), outlier_dates[3] - pd.Timedelta(days=1)),
        outlier_dates[4]: (outlier_dates[4] - pd.DateOffset(months=6), outlier_dates[4] - pd.Timedelta(days=1))
    }

    def redistribute_cases(aggregated_data, outlier_dates, redistribution_periods, threshold=0.95):
        for index, outlier_date in enumerate(outlier_dates):
            period = redistribution_periods[outlier_date]
            start_date, end_date = period
            period_data = aggregated_data[(aggregated_data.index >= start_date) & (aggregated_data.index <= end_date)]
            
            # Calculate the threshold for the period
            threshold_affected = int(period_data['Animals affected'].quantile(threshold))
            threshold_died = int(period_data['Animals died'].quantile(threshold))
            
            # Identify excess cases
            excess_affected = int(aggregated_data.loc[outlier_date, 'Animals affected']) - threshold_affected
            excess_died = int(aggregated_data.loc[outlier_date, 'Animals died']) - threshold_died
            
            if excess_affected > 0:
                redistribute_column_cases(aggregated_data, period, outlier_date, excess_affected, 'Animals affected')
            if excess_died > 0:
                redistribute_column_cases(aggregated_data, period, outlier_date, excess_died, 'Animals died')
        
        return aggregated_data

    def redistribute_column_cases(aggregated_data, period, outlier_date, excess_cases, column):
        start_date, end_date = period
        redistribution_data = aggregated_data[(aggregated_data.index >= start_date) & (aggregated_data.index <= end_date)].copy()
        
        # Calculate the proportional distribution
        total_past_cases = redistribution_data[column].sum()
        if total_past_cases == 0:
            return  # Skip if there are no past cases to redistribute into

        redistribution_data.loc[:, 'proportion'] = redistribution_data[column] / total_past_cases
        
        # Redistribute cases proportionally
        cases_distributed = 0
        for index, row in redistribution_data.iterrows():
            cases_to_add = int(row['proportion'] * excess_cases)
            aggregated_data.loc[index, column] += cases_to_add
            cases_distributed += cases_to_add
        
        # Handle remaining cases
        remaining_cases = excess_cases - cases_distributed
        for index in redistribution_data.index:
            if remaining_cases <= 0:
                break
            aggregated_data.loc[index, column] += 1
            remaining_cases -= 1

        # Adjust the outlier date count
        aggregated_data.loc[outlier_date, column] -= excess_cases

    # Perform the redistribution
    aggregated_data = redistribute_cases(aggregated_data, outlier_dates, redistribution_periods)

    # Verify final totals
    affected_sum = aggregated_data['Animals affected'].sum()
    died_sum = aggregated_data['Animals died'].sum()
    row_count = len(aggregated_data)

    assert affected_sum == original_totals['Animals affected'], f"Mismatch in Animals affected totals: expected {original_totals['Animals affected']}, got {affected_sum}"
    assert died_sum == original_totals['Animals died'], f"Mismatch in Animals died totals: expected {original_totals['Animals died']}, got {died_sum}"
    assert row_count == original_totals['rows'], f"Row count mismatch: expected {original_totals['rows']}, got {row_count}"

    print()
    print("Redistribution (beta) complete!")
    print(f"Total cases: {len(aggregated_data)}")
    print(f"Animals affected: {aggregated_data['Animals affected'].sum()}")
    print(f"Animals died: {aggregated_data['Animals died'].sum()}")

    # Drop the original columns before displaying the data
    aggregated_data.drop(columns=['Animals affected original', 'Animals died original'], inplace=True)

    # Output the modified dataset
    aggregated_data.to_csv(OUTPUT_EVVET_ADJ)
except Exception as e:
    print(f"Failed to load data: {e}")